In [34]:
import os
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score


In [35]:
cwd = os.getcwd()

df_train = pd.read_csv(os.path.join(cwd, 'Preprocessed_Train.csv'))
df_test = pd.read_csv(os.path.join(cwd, 'Preprocessed_Test.csv'))


In [36]:
X_train, y_train = df_train['comments'], df_train['subreddits']

In [37]:
df_test = df_test['comments']

#### TFIDF VECTORIZER

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=None, smooth_idf=True, sublinear_tf=True)



Encoder = LabelEncoder()

y_train = Encoder.fit_transform(y_train)

tfidf_vectorizer.fit(X_train)
X_train = tfidf_vectorizer.transform(X_train)


df_test = tfidf_vectorizer.transform(df_test)



In [39]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model





In [40]:
import warnings
warnings.simplefilter('ignore')

In [41]:
clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=1000)
clf2 = linear_model.SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, n_iter=100, random_state=None, learning_rate='optimal', early_stopping=False, validation_fraction=0.1, n_iter_no_change=5)
clf3 = naive_bayes.MultinomialNB(alpha=0.2, fit_prior=True, class_prior=None)


## Kaggle Algorithm

In [42]:
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_predict

In [43]:
from mlxtend.classifier import EnsembleVoteClassifier

eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[1,1,1],voting='hard')


labels = ['Logistic Regression', 'Stochastc GD', 'Naive Bayes', 'Ensemble']
for clf, label in zip([clf1, clf2, clf3, eclf], labels):

    scores = model_selection.cross_val_score(clf, X_train , y_train,  
                                              cv=5, 
                                              scoring='accuracy')
    
    print("Accuracy: %0.6f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))
    


Accuracy: 0.545108 (+/- 0.00) [Logistic Regression]
Accuracy: 0.550951 (+/- 0.00) [Stochastc GD]
Accuracy: 0.566337 (+/- 0.00) [Naive Bayes]
Accuracy: 0.571151 (+/- 0.00) [Ensemble]


In [44]:
eclf.fit(X_train, y_train)

EnsembleVoteClassifier(clfs=[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False), SGDClassifier(alpha=0.0...ction=0.1, verbose=0, warm_start=False), MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)],
            refit=True, verbose=0, voting='hard', weights=[1, 1, 1])

In [45]:
eclf.predict(df_test)

array([ 5,  8,  4, ...,  1, 10, 19])

In [46]:
eclf_pred = eclf.predict(df_test)

In [47]:
eclf_pred = Encoder.inverse_transform(eclf_pred)

## END HERE

In [27]:
df_test = pd.read_csv(os.path.join(cwd, 'Preprocessed_Test.csv'))
columns_new = ['id', 'category']
df_submit = pd.DataFrame(df_test, columns=columns_new)

df_preds = pd.DataFrame(eclf_pred)
df_preds['category'] = eclf_pred

df_submit['category']= df_preds['category']
df_submit['id'] = df_test['id']

In [28]:
df_submit.to_csv('C:/Users/adamb/Desktop/COMP551 Applied Machine Learning/Project2/code/check_on_before_preprocessing.csv', index=False)